## Prepare model

### Imports & Definitions

In [1]:
import warnings

warnings.filterwarnings(
    "ignore",
    message="torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.",
)

warnings.filterwarnings(
    "ignore",
    message="torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.",
)

warnings.filterwarnings(
    "ignore",
    message="You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset",
)

warnings.filterwarnings(
    "ignore",
    message="`do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.",
)

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import sys
sys.path.append("../../utils")
from definitions import *

In [4]:
CACHE_DIR = Path("../../../../cache/")
DATASET_DIR = Path("/home/jupyter/mnt/datasets/diplomas/russian_dataset/")

In [6]:
MODEL_EXPERIMENT_NAME = "learnt_16k"

### Load model & tokenizer

In [7]:
import sys
sys.path.append("../../../../LongLoRA-diploma-research")

In [8]:
import sys
import io
import os
import copy
import json
import math
import logging
import pandas as pd
from dataclasses import dataclass, field
from typing import Dict, Optional, Sequence

import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer, DataCollatorForLanguageModeling
from llama_attn_replace_sft import replace_llama_attn
from gptneox_attn_replace import replace_gpt_neox_attn
from peft import LoraConfig, get_peft_model
from torch.distributed import barrier


MODEL_MAX_LENGTH = 64000
model_name = LLAMA_2_7B

replace_llama_attn(True, False)

# Set RoPE scaling factor
config = transformers.AutoConfig.from_pretrained(
    HUGGINGFACE_MODEL_TO_REPO[model_name],
    cache_dir=CACHE_DIR,
)

orig_rope_scaling = getattr(config, "rope_scaling", None)
if orig_rope_scaling is None:
    orig_rope_scaling = {"factor": 1}
orig_rope_scaling_factor = orig_rope_scaling["factor"] if "factor" in orig_rope_scaling.keys() else 1
orig_ctx_len = getattr(config, "max_position_embeddings", None)
if orig_ctx_len:
    orig_ctx_len *= orig_rope_scaling_factor
    if MODEL_MAX_LENGTH > orig_ctx_len:
        scaling_factor = float(math.ceil(MODEL_MAX_LENGTH / orig_ctx_len))
        config.rope_scaling = {"type": "linear", "factor": scaling_factor}

print("Created config")

# Load model and tokenizer
model = transformers.AutoModelForCausalLM.from_pretrained(
    HUGGINGFACE_MODEL_TO_REPO[model_name], 
    cache_dir=CACHE_DIR, 
    config=config,
    torch_dtype=torch.bfloat16,
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

print("Loaded model")

tokenizer = transformers.AutoTokenizer.from_pretrained(
    HUGGINGFACE_MODEL_TO_REPO[model_name],
    cache_dir=CACHE_DIR,
    model_max_length=MODEL_MAX_LENGTH,
    padding_side="right",
    use_fast=True,
)

print("Loaded tokenizer")

IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"


special_tokens_dict = dict()
if tokenizer.pad_token is None:
    special_tokens_dict["pad_token"] = DEFAULT_PAD_TOKEN
if tokenizer.eos_token is None:
    special_tokens_dict["eos_token"] = DEFAULT_EOS_TOKEN
if tokenizer.bos_token is None:
    special_tokens_dict["bos_token"] = DEFAULT_BOS_TOKEN
if tokenizer.unk_token is None:
    special_tokens_dict["unk_token"] = DEFAULT_UNK_TOKEN

# if training_args.low_rank_training:
#     if model_args.model_type == "gpt-neox":
#         # added `dense` to match with llama as the basic LoRA would only target 'query_key_value'
#         targets = ["query_key_value", "dense"]
#     else:
#         targets=["q_proj", "k_proj", "v_proj", "o_proj"]

#     config = LoraConfig(
#         r=8,
#         lora_alpha=16,
#         target_modules=targets,
#         lora_dropout=0,
#         bias="none",
#         task_type="CAUSAL_LM",
#     )
#     model = get_peft_model(model, config)
#     # enable trainable params
#     [p.requires_grad_() for n, p in model.named_parameters() if any([k in n for k in training_args.trainable_params.split(",")])]

# model.config.use_cache = False         # required for gradient checkpointing
# model.enable_input_require_grads()     # required for gradient checkpointing
# model.gradient_checkpointing_enable()  # enable gradient checkpointing

# print("Prepared model to learn")

###

# model_name = LLAMA_2_7B
# model = AutoModelForCausalLM.from_pretrained(
#     HUGGINGFACE_MODEL_TO_REPO[model_name], 
#     cache_dir=CACHE_DIR, 
#     device_map='auto'
# )
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model.to(device)

# MODEL_MAX_LENGTH = 16384

# tokenizer = AutoTokenizer.from_pretrained(
#     HUGGINGFACE_MODEL_TO_REPO[model_name], 
#     cache_dir=CACHE_DIR, 
#     model_max_length=MODEL_MAX_LENGTH,
#     padding_side="right",
#     use_fast=True
# )

# IGNORE_INDEX = -100
# DEFAULT_PAD_TOKEN = "[PAD]"
# DEFAULT_EOS_TOKEN = "</s>"
# DEFAULT_BOS_TOKEN = "<s>"
# DEFAULT_UNK_TOKEN = "<unk>"

# def smart_tokenizer_and_embedding_resize(
#     special_tokens_dict: Dict,
#     tokenizer: transformers.PreTrainedTokenizer,
#     model: transformers.PreTrainedModel,
# ):
#     """Resize tokenizer and embedding.

#     Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
#     """
#     num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
#     model.resize_token_embeddings(len(tokenizer))

#     if num_new_tokens > 0:
#         input_embeddings = model.get_input_embeddings().weight.data
#         output_embeddings = model.get_output_embeddings().weight.data

#         input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
#         output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

#         input_embeddings[-num_new_tokens:] = input_embeddings_avg
#         output_embeddings[-num_new_tokens:] = output_embeddings_avg

# special_tokens_dict = dict()
# if tokenizer.pad_token is None:
#     special_tokens_dict["pad_token"] = DEFAULT_PAD_TOKEN
# if tokenizer.eos_token is None:
#     special_tokens_dict["eos_token"] = DEFAULT_EOS_TOKEN
# if tokenizer.bos_token is None:
#     special_tokens_dict["bos_token"] = DEFAULT_BOS_TOKEN
# if tokenizer.unk_token is None:
#     special_tokens_dict["unk_token"] = DEFAULT_UNK_TOKEN

# smart_tokenizer_and_embedding_resize(
#     special_tokens_dict=special_tokens_dict,
#     tokenizer=tokenizer,
#     model=model,
# )

2024-05-10 16:18:18.989530: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Created config


Loading checkpoint shards: 100%|██████████| 2/2 [03:40<00:00, 110.13s/it]


Loaded model
Loaded tokenizer

Using pad_token, but it is not set yet.


In [9]:
    
def smart_tokenizer_and_embedding_resize(
    special_tokens_dict: Dict,
    tokenizer: transformers.PreTrainedTokenizer,
    model: transformers.PreTrainedModel,
):
    """Resize tokenizer and embedding.

    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))

    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
        output_embeddings = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
        output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings[-num_new_tokens:] = input_embeddings_avg
        output_embeddings[-num_new_tokens:] = output_embeddings_avg
        
smart_tokenizer_and_embedding_resize(
    special_tokens_dict=special_tokens_dict,
    tokenizer=tokenizer,
    model=model,
)

In [10]:
from peft import PeftModel

model_id = "nvdenisov2002/llama-longLoRA-v3-16k-5000-samples-220-iterations"
peft_model = PeftModel.from_pretrained(model, model_id)

## Inference model on tasks

In [11]:
first_k = 5 # int(1e5)
INF = int(1e7)

In [12]:
first_k, INF

(5, 10000000)

### Inference model on asessors questions

In [13]:
from llm_helpers_fixed_tokenizer import get_some_model_result

test_df = pd.read_csv(ARTIFACTS_DIR_PATH.joinpath("diplomas_abstracts/mcs_raw_learnt_abstract_learnt8k.csv"))

new_rows = []
for _, row in tqdm(test_df[:first_k].iterrows(), total=len(test_df[:first_k]), desc="Rows..."):
    new_row = copy.deepcopy(row)
    new_row[MODEL_EXPERIMENT_NAME] = get_some_model_result(peft_model, tokenizer, row, device, diploma_prefix_len=INF)
    new_rows.append(new_row)
new_df = pd.DataFrame(new_rows)
new_df.head()

Rows...:   0%|          | 0/5 [00:00<?, ?it/s]

Texts...:   0%|          | 0/1 [00:00<?, ?it/s]

Texts...:   0%|          | 0/1 [00:00<?, ?it/s]

/home/jupyter/work/resources/long_context_LLMs/src/notebooks/metrics_calculation/../../utils/llm_helpers_fixed_tokenizer.py:121: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_id = torch.tensor(res, dtype=torch.int)
/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 13700, but `max_length` is set to 4096. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/home/jupyter/work/resources/long_context_LLMs

Texts...:   0%|          | 0/1 [00:00<?, ?it/s]

...

/home/jupyter/work/resources/long_context_LLMs/src/notebooks/metrics_calculation/../../utils/llm_helpers_fixed_tokenizer.py:121: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_id = torch.tensor(res, dtype=torch.int)
/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 10684, but `max_length` is set to 4096. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/home/jupyter/work/resources/long_context_LLMs

Texts...:   0%|          | 0/1 [00:00<?, ?it/s]

Texts...:   0%|          | 0/1 [00:00<?, ?it/s]

/home/jupyter/work/resources/long_context_LLMs/src/notebooks/metrics_calculation/../../utils/llm_helpers_fixed_tokenizer.py:121: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_id = torch.tensor(res, dtype=torch.int)
/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 15467, but `max_length` is set to 4096. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
/home/jupyter/work/resources/long_context_LLMs

,Unnamed: 0.1,Unnamed: 0,id,year,diploma,abstract,study_field,degree,original_diploma_extension,raw_model,learnt,learnt_8k,learnt_16k
0,0,12,45042,2023,АЙВАЗЬЯН Аршак Владимирович\nВыпускная квалифи...,В этой работе мы строим правую трансферную мод...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В работе рассматривается модельная структура н...,В данной работе рассматривается модельная стру...,В работе рассматриваются алгебраические теории...,<s>
1,1,25,45043,2023,Санкт-Петербургский государственный университе...,"Пусть 𝐾 выпуклое тело в ℝ^𝑛. Определим 𝑑𝑛,𝑛−1(...",MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,Плотность решетки трансляций - это минимальная...,В работе рассматриваются плотности решеток тра...,В,Response
2,2,37,45044,2023,Санкт-Петербургский государственный университе...,Работа посвящена повышению производительности ...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В данной работе представлены результаты исслед...,В работе рассматривается задача булевой выполн...,In this work we propose a method for improving...,The
3,3,101,45046,2023,Санкт-Петербургский государственный университе...,В работе мы обобщаем результаты об энергии нат...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В работе рассматриваются классы случайных проц...,В данной работе рассматривается энергетически-...,В работе мы обобщаем результаты об энергии нат...,.
4,4,152,45047,2023,Санкт–Петербургский государственный университе...,В рамках данной работы рассматривается подход ...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В данной работе рассматривается задача настраи...,В данной работе рассматривается задача добавле...,В работе рассматривается применение добавления...,<s>


In [36]:
new_df.head(30)

,Unnamed: 0.1,id,year,diploma,abstract,study_field,degree,original_diploma_extension,raw_model,learnt,learnt_8k,learnt_64k
Unnamed: 0,,,,,,,,,,,,
12,0,45042,2023,АЙВАЗЬЯН Аршак Владимирович\nВыпускная квалифи...,В этой работе мы строим правую трансферную мод...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В работе рассматривается модельная структура н...,В данной работе рассматривается модельная стру...,В работе рассматриваются алгебраические теории...,\n
25,1,45043,2023,Санкт-Петербургский государственный университе...,"Пусть 𝐾 выпуклое тело в ℝ^𝑛. Определим 𝑑𝑛,𝑛−1(...",MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,Плотность решетки трансляций - это минимальная...,В работе рассматриваются плотности решеток тра...,В,On
37,2,45044,2023,Санкт-Петербургский государственный университе...,Работа посвящена повышению производительности ...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В данной работе представлены результаты исслед...,В работе рассматривается задача булевой выполн...,In this work we propose a method for improving...,","
101,3,45046,2023,Санкт-Петербургский государственный университе...,В работе мы обобщаем результаты об энергии нат...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В работе рассматриваются классы случайных проц...,В данной работе рассматривается энергетически-...,В работе мы обобщаем результаты об энергии нат...,\n
152,4,45047,2023,Санкт–Петербургский государственный университе...,В рамках данной работы рассматривается подход ...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В данной работе рассматривается задача настраи...,В данной работе рассматривается задача добавле...,В работе рассматривается применение добавления...,\n
180,5,45048,2023,Санкт-Петербургский государственный университе...,"В работе доказывается, что типичное сохраняюще...",MATHEMATICS AND COMPUTER SCIENCE,MASTER'S STUDIES,.pdf,В данной работе рассматривается понятие «scali...,Обозначим группу \(G\) и действие \(G\) на про...,В данной работе рассматривается проблема сущес...,\n
196,6,45049,2023,Санкт–Петербургский государственный университе...,В этой работе описывается разработка генератор...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В данной работе представлена программная реали...,В данной работе рассмотрены методы реализации ...,В данной работе предлагается генератор языка з...,рован
205,7,45050,2023,Санкт-Петербургский государственный университе...,В данной работе разработан метод вариативной и...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В работе рассматривается задача идентификации ...,Данная выпускная квалификационная работа посвя...,В работе рассмотрена задача идентификации масс...,ва
207,8,45051,2023,Санкт–Петербургский государственный университе...,В данной работе представлена реализация фаззер...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,"В выпускной квалификационной работе ""Разработк...",В выпускной квалификационной работе рассматрив...,В работе рассматривается фаззинг программ на я...,\n


In [30]:
new_df.to_csv(ARTIFACTS_DIR_PATH.joinpath(f"diplomas_abstracts/baselines_with_{MODEL_EXPERIMENT_NAME}.csv"))

### Inference model on MERA

In [ ]:
from mera_helpers import construct_prompt
from path_helpers import get_dataset_path, get_metric_dir_path
from llm_helpers import calculate_token_interest_probs, get_answer

for name, dataset_meta in tqdm(HUGGINGFACE_NAME_TO_DATASET.items(), desc="Datasets..."):
    for subset, split in tqdm(zip(dataset_meta["subsets"], dataset_meta["splits"]), total=len(dataset_meta["splits"]), desc="Splits..."):
        path = get_dataset_path(subset, name, split)
        dataset = load_from_disk(path)
        probs_list = []
        a_list = []
        for row in tqdm(list(dataset)[:first_k], desc="Rows..."):
            q = construct_prompt(row)
            probs = calculate_token_interest_probs(q, tokenizer, peft_model)
            probs_list.append({
                "probs": probs,
                "meta": row["meta"],
            })
            a = get_answer(probs)
            a_list.append({
                "answer": a,
                "meta": row["meta"],
            })
        print(a_list)
        metric_dir_path = get_metric_dir_path(MODEL_EXPERIMENT_NAME, subset, name, split)
        print(metric_dir_path)
        metric_dir_path.mkdir(exist_ok=True, parents=True)
        with open(metric_dir_path.joinpath("probs.jsonl"), "w") as f:
            json.dump(probs_list, f, ensure_ascii=False, indent=2)
        with open(metric_dir_path.joinpath("answers.jsonl"), "w") as f:
            json.dump(a_list, f, ensure_ascii=False, indent=2)

### Inference & eval model on landmark passkey

In [52]:
! ls ../../../../LongLoRA-diploma-research

DATA_LICENSE
LICENSE
README.md
WEIGHT_LICENSE
__pycache__
benchmarks
checkpoints
demo.py
ds_configs
eval.py
eval_distributed.py
fine-tune.py
get_trainable_weights.py
gptneox_attn_replace.py
imgs
inference-qlora.py
inference.py
llama_attn_replace.py
llama_attn_replace_sft.py
llama_replaced_attn_128k_logging_dir
llama_replaced_attn_128k_output_dir
llama_replaced_attn_16k_without_checkpoints_logging_dir
llama_replaced_attn_16k_without_checkpoints_new_tokenize_logging_dir
llama_replaced_attn_16k_without_checkpoints_new_tokenize_output_dir
llama_replaced_attn_16k_without_checkpoints_output_dir
llama_replaced_attn_64k_without_checkpoints_logging_dir
llama_replaced_attn_64k_without_checkpoints_output_dir
llama_replaced_attn_logging_dir
llama_replaced_attn_output_dir
load_dataset.ipynb
merge_lora_weights_and_save_hf_model.py
passkey_retrivial.py
pdf2txt
requirements.txt
run_learning_saiga_mistral_lora_7b.ipynb
run_streaming_llama_longalpaca.py
streaming_llm
supervised-fine-tune-qlora.py
superv

In [14]:
! python3 ../../../../LongLoRA-diploma-research/passkey_retrivial.py \
        --context_size 64000 \
        --base_model ../../../../LongLoRA-diploma-research/llama_replaced_attn_64k_without_checkpoints_output_dir \
        --max_tokens 32768 \
        --interval 10000

^C


TypeError: cannot unpack non-iterable NoneType object

In [38]:
peft_model.device

device(type='cuda', index=0)

In [44]:
import torch

import os
import random
import re
import requests
from tqdm import tqdm, trange
import json
from collections import defaultdict
import sys

def generate_prompt(n_garbage):
    """Generates a text file and inserts an execute line at a random position."""
    n_garbage_prefix = random.randint(0, n_garbage)
    n_garbage_suffix = n_garbage - n_garbage_prefix
    
    task_description = "There is an important info hidden inside a lot of irrelevant text. Find it and memorize them. I will quiz you about the important information there."
    garbage = "The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again."
    garbage_inf = " ".join([garbage] * 2000)
    assert len(garbage_inf) >= n_garbage
    garbage_prefix = garbage_inf[:n_garbage_prefix]
    garbage_suffix = garbage_inf[:n_garbage_suffix]
    pass_key = random.randint(1, 50000)
    information_line = f"The pass key is {pass_key}. Remember it. {pass_key} is the pass key."
    final_question = "What is the pass key? The pass key is"
    lines = [
        task_description,
        garbage_prefix,
        information_line,
        garbage_suffix,
        final_question
    ]
    return "\n".join(lines), pass_key

In [50]:
def test_model(prompt_text, pass_key, model_name, pipes):
    response = pipes[model_name](prompt_text,num_return_sequences=1, max_new_tokens=10)[0]["generated_text"][len(prompt_text):]
    assert f"The pass key is {pass_key}" in prompt_text

    try:
        pass_key = int(re.search(r'\d+', response).group())
    except:
        pass_key = response[:20]
    
    return pass_key

In [51]:
n_values=[5, 14000, 18000]
num_tokens_in_prompt = defaultdict(list)
for n in tqdm(n_values, desc="n_values..."):

    correct_count = {x: 0 for x in models}

    n_results = {x: [] for x in models}
    for i in trange(2, desc="Tests..."):
        # print(f"\nRunning test {i + 1}/{num_tests} for n = {n}...")
        prompt_text, pass_key = generate_prompt(n)



        for model_name in models:
            if pipes[model_name] is None:
                continue
            num_tokens = len(pipes[model_name].tokenizer.encode(prompt_text))
            num_tokens_in_prompt[f"{model_name}_{n}"].append(num_tokens)

            # print("Number of tokens in this prompt: ", num_tokens)
            print(prompt_text, pass_key, model_name)
            model_output = test_model(prompt_text, pass_key, model_name, pipes)
            # print(f"Expected number in the prompt: {pass_key}, {model_name} output: {model_output}")

            if pass_key == model_output:
                correct_count[model_name] += 1
                n_results[model_name].append(1)
                # print("Success!")
            else:
                n_results[model_name].append(0)

Tests...:   0%|          | 0/2 [00:00<?, ?it/s]/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
n_values...:   0%|          | 0/3 [00:00<?, ?it/s]

There is an important info hidden inside a lot of irrelevant text. Find it and memorize them. I will quiz you about the important information there.
Th
The pass key is 595. Remember it. 595 is the pass key.
The
What is the pass key? The pass key is 595 learnt_64k


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [16]:
from landmark_run_test import test_passkey_full, load_pipes
from transformers import pipeline


models = [MODEL_EXPERIMENT_NAME]
pipe = pipeline("text-generation", model=peft_model, tokenizer=tokenizer, device='cuda')
pipes = {MODEL_EXPERIMENT_NAME: pipe}
test_passkey_full(pipes, models, n_values=[14000, 18000])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartForCausalLM', 'Prophe

models = ['learnt_64k']


n_values...:   0%|          | 0/2 [00:00<?, ?it/s]

Tests...:   0%|          | 0/50 [00:00<?, ?it/s]

/home/jupyter/work/resources/long_context_LLMs/src/notebooks/metrics_calculation/../../../../LongLoRA-diploma-research/llama_attn_replace_sft.py:39: UserWarning: This function should be used just for training as it may exhibit reduced inference performance. For inference, please use forward_flashattn_inference.
  warnings.warn("This function should be used just for training as it may exhibit reduced inference performance. For inference, please use forward_flashattn_inference.")
/home/jupyter/work/resources/long_context_LLMs/src/notebooks/metrics_calculation/../../../../LongLoRA-diploma-research/llama_attn_replace_sft.py:39: UserWarning: This function should be used just for training as it may exhibit reduced inference performance. For inference, please use forward_flashattn_inference.
  warnings.warn("This function should be used just for training as it may exhibit reduced inference performance. For inference, please use forward_flashattn_inference.")
/home/jupyter/work/resources/long_

RuntimeError: stack expects each tensor to be equal size, but got [1, 32, 1, 128] at entry 0 and [1, 32, 3805, 128] at entry 1

## Evaluate model results on tasks

### Evaluate asessors questions

In [31]:
new_df = new_df.set_index("Unnamed: 0")
new_df

,Unnamed: 0.1,id,year,diploma,abstract,study_field,degree,original_diploma_extension,raw_model,learnt,learnt_8k,learnt_64k
Unnamed: 0,,,,,,,,,,,,
12,0,45042,2023,АЙВАЗЬЯН Аршак Владимирович\nВыпускная квалифи...,В этой работе мы строим правую трансферную мод...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В работе рассматривается модельная структура н...,В данной работе рассматривается модельная стру...,В работе рассматриваются алгебраические теории...,\n
25,1,45043,2023,Санкт-Петербургский государственный университе...,"Пусть 𝐾 выпуклое тело в ℝ^𝑛. Определим 𝑑𝑛,𝑛−1(...",MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,Плотность решетки трансляций - это минимальная...,В работе рассматриваются плотности решеток тра...,В,On
37,2,45044,2023,Санкт-Петербургский государственный университе...,Работа посвящена повышению производительности ...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В данной работе представлены результаты исслед...,В работе рассматривается задача булевой выполн...,In this work we propose a method for improving...,","
101,3,45046,2023,Санкт-Петербургский государственный университе...,В работе мы обобщаем результаты об энергии нат...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В работе рассматриваются классы случайных проц...,В данной работе рассматривается энергетически-...,В работе мы обобщаем результаты об энергии нат...,\n
152,4,45047,2023,Санкт–Петербургский государственный университе...,В рамках данной работы рассматривается подход ...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В данной работе рассматривается задача настраи...,В данной работе рассматривается задача добавле...,В работе рассматривается применение добавления...,\n
...,...,...,...,...,...,...,...,...,...,...,...,...
1062,65,45131,2023,Санкт-Петербургский государственный университе...,В настоящей работе были рассмотрены две задачи...,MATHEMATICS AND COMPUTER SCIENCE,MASTER'S STUDIES,.pdf,"В работе рассматриваются морфизмы, порождающие...",В работе рассматриваются некоторые гипотезы о ...,В данной работе изучаются некоторые гипотезы о...,вид
1064,66,45132,2023,Санкт-Петербургский государственный университе...,"В данной работе исследовались MS/MS спектры, с...",MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,Данная работа посвящена разработке алгоритмов ...,Данная работа посвящена поиску и классификации...,Данная работа посвящена исследованию поведения...,\n
1079,67,45133,2023,Санкт-Петербургский государственный университе...,В выпускной работе описываются триангуляции ве...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В работе рассматриваются асимптотики числа три...,Автор производит подсчёт асимптотики числа три...,"Рассматриваются выпуклые триангуляции, то есть...",тре


In [32]:
from mera_helpers import construct_prompt
from llm_helpers import calculate_token_interest_probs, get_answer


with open(ARTIFACTS_DIR_PATH.joinpath("datasets/diplomas_asessors_questions/mcs_df_human_filled_processed.json"), "r") as f:
    asessors_dataset = json.load(f)

a_list = []
probs_list = []
for row in tqdm(asessors_dataset[:first_k], desc="Rows..."):
    x = copy.deepcopy(row)
    x['inputs']['context'] = new_df[MODEL_EXPERIMENT_NAME].loc[int(x['meta']['id'])] # df["diploma"].iloc[int(x['meta']['id'])]
    q = construct_prompt(x)
    probs = calculate_token_interest_probs(q, tokenizer, peft_model)
    probs_list.append({
        "probs": probs,
        "meta": row["meta"],
    })
    a = get_answer(probs)
    a_list.append({
        "answer": a,
        "meta": row["meta"],
    })
metric_dir_path = ARTIFACTS_DIR_PATH.joinpath(f"metrics/diplomas_asessors_questions/{MODEL_EXPERIMENT_NAME}/{LLAMA_2_7B}/") # get_metric_dir_path(baseline_model, subset, name, split)
metric_dir_path.mkdir(exist_ok=True, parents=True)
with open(metric_dir_path.joinpath("probs.jsonl"), "w") as f:
    json.dump(probs_list, f, ensure_ascii=False, indent=2)
with open(metric_dir_path.joinpath("answers.jsonl"), "w") as f:
    json.dump(a_list, f, ensure_ascii=False, indent=2)

Rows...:   0%|          | 0/70 [00:00<?, ?it/s]

/home/jupyter/work/resources/long_context_LLMs/src/notebooks/metrics_calculation/../../../../LongLoRA-diploma-research/llama_attn_replace_sft.py:39: UserWarning: This function should be used just for training as it may exhibit reduced inference performance. For inference, please use forward_flashattn_inference.
  warnings.warn("This function should be used just for training as it may exhibit reduced inference performance. For inference, please use forward_flashattn_inference.")
/home/jupyter/work/resources/long_context_LLMs/src/notebooks/metrics_calculation/../../../../LongLoRA-diploma-research/llama_attn_replace_sft.py:39: UserWarning: This function should be used just for training as it may exhibit reduced inference performance. For inference, please use forward_flashattn_inference.
  warnings.warn("This function should be used just for training as it may exhibit reduced inference performance. For inference, please use forward_flashattn_inference.")
/home/jupyter/work/resources/long_

In [33]:
rows = []
pred_by_model = dict()
for baseline_model in HUGGINFACE_BASELINE_MODELS:
    splits = ["abstract", "empty", "diploma"]
    if baseline_model == LLAMA_2_7B:
        splits.extend(["learnt", "learnt_8k", MODEL_EXPERIMENT_NAME])
    for split in splits:
        metric_path = ARTIFACTS_DIR_PATH.joinpath(f"metrics/diplomas_asessors_questions/{split}/{baseline_model}/") # get_metric_dir_path(baseline_model, subset, name, split)
        with open(metric_path.joinpath("answers.jsonl"), "r") as f:
            answers = json.load(f)
        pred = [x["answer"] for x in answers[:first_k]]
        pred_by_model[f"{split}_{baseline_model}"] = pred
        true = [x["outputs"] for x in asessors_dataset[:first_k]]
        rows.append({
            "model": baseline_model,
            "subset": "asessors_questions",
            "split": f"{split}_8000" if split == "diploma" else split,
            "accuracy_score": accuracy_score(true, pred),
        })
asessors_df = pd.DataFrame(rows)
asessors_df

,model,subset,split,accuracy_score
0,mistral-7b,asessors_questions,abstract,0.285714
1,mistral-7b,asessors_questions,empty,0.185714
2,mistral-7b,asessors_questions,diploma_8000,0.257143
3,llama-2-7b,asessors_questions,abstract,0.285714
4,llama-2-7b,asessors_questions,empty,0.228571
5,llama-2-7b,asessors_questions,diploma_8000,0.200000
6,llama-2-7b,asessors_questions,learnt,0.228571
7,llama-2-7b,asessors_questions,learnt_8k,0.271429
8,llama-2-7b,asessors_questions,learnt_64k,0.228571


In [34]:
asessors_df.to_csv(METRICS_DIR_PATH.joinpath(f"asessors_with_{MODEL_EXPERIMENT_NAME}.csv"))

### Evaluate MERA

In [ ]:
rows = []
for baseline_model in HUGGINFACE_BASELINE_MODELS.union([VIKHR_7B, SAIGA_MISTRAL_7B_LORA]).union([MODEL_EXPERIMENT_NAME]):
    for name, dataset_meta in HUGGINGFACE_NAME_TO_DATASET.items():
        for subset, split in zip(dataset_meta["subsets"], dataset_meta["splits"]):
            dataset_path = get_dataset_path(subset, name, split)
            dataset = load_from_disk(dataset_path)
            metric_path = get_metric_dir_path(baseline_model, subset, name, split)
            with open(metric_path.joinpath("answers.jsonl"), "r") as f:
                answers = json.load(f)
            pred = [x["answer"] for x in answers[:first_k]]
            true = [x["outputs"] for x in list(dataset)[:first_k]]
            rows.append({
                "model": baseline_model,
                "subset": subset,
                "split": split,
                "accuracy_score": accuracy_score(true, pred),
            })
df = pd.DataFrame(rows)
df

In [ ]:
df.to_csv(METRICS_DIR_PATH.joinpath(f"ru_metrics_with_{MODEL_EXPERIMENT_NAME}.csv"))